In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import generate_df as G
from IPython.display import display, HTML
from tabulate import tabulate
from unidecode import unidecode

In [4]:
# spark = SparkSession.builder.appName("finance_data_processing")\
# .config("spark.jars","/home/daiane/Downloads/postgresql-42.7.3.jar").getOrCreate()

df_demonstracao_result_ = G.table_to_df("finance_raw_data", "demonstracao_resultado_receita_lucro", spark)
df_demonstracao_result_.persist()

df_temp = df_demonstracao_result_.withColumn("data", F.to_date(F.col("data"), "MM/yyyy"))

last_date = df_temp.agg(F.max("data")).collect()[0][0]

last_date_str = last_date.strftime("%m/%Y")


df_demonstracao_result = df_demonstracao_result_.filter(F.col("data") == last_date_str)
df_demonstracao_result.persist()

print(last_date)
print(last_date_str)


dfSample_demonstracao_result = df_demonstracao_result.sample(False, 0.1).limit(30)
table = tabulate(dfSample_demonstracao_result.collect(), headers=dfSample_demonstracao_result.columns, tablefmt='html')

display(HTML(table))
print(dfSample_demonstracao_result.count())
# dfSample_demonstracao_result.printSchema()



24/07/17 19:00:40 WARN TaskSetManager: Stage 11 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:00:45 WARN TaskSetManager: Stage 12 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


2024-03-01
03/2024


24/07/17 19:00:45 WARN TaskSetManager: Stage 15 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:00:46 WARN TaskSetManager: Stage 16 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:00:46 WARN TaskSetManager: Stage 17 contains a task of very large size (12468 KiB). The maximum recommended task size is 1000 KiB.


instituicao,codigo,conglomerado,codigo_conglomerado_financeiro,codigo_conglomerado_prudencial,tcb,tc,ti,cidade,uf,data,rendas_operacoes_credito_a1,rendas_operacoes_arrendamento_mercantil_a2,rendas_operacoes_com_tvm_a3,rendas_operacoes_instrumentos_financeiros_derivativos_a4,resultado_operacoes_cambio_a5,rendas_aplicacoes_compulsorias_a6,receitas_intermediacao_financeira_soma_a1_a2_a3_a4_a5_a6,despesas_captacao_b1,despesas_obrigacoes_por_emprestimos_e_repasses_b2,despesas_operacoes_arrendamento_mercantil_b3,resultado_operacoes_cambio_b4,resultado_de_provisao_para_creditos_dificil_liquidacao_b5,despesas__intermediacao_financeira_soma_b1_b2_b3_b4_b5,resultao_intermediacao_financeira_c_soma_a_b,rendas_prestacao_servico_d1,rendas_tarifas_bancarias_d2,despesas_de_pessoal_d3,despesas_administrativas_d4,despesas_tributarias_d5,resultado_de_participacoes_d6,outras_receitas_operacionais_d7,outras_receitas_operacionais_d8,soma_d1_d2_d3_d4_d5_d6_d7_d8,resultado_operacional_soma_c_d,resultado_nao_operacional_f,g,imposto_renda_contribuicao_social_h,participacao_lucros_i,lucro_liquido_soma_g_h_i,juros_capital_social_cooperativas_k
ITAÚ UNIBANCO S.A.,60701190,ITAU - PRUDENCIAL,10069,80099,b1,2,8,SAO PAULO,SP,03/2024,19.599.269,0,24.344.951,3.119.430,167.561,2.844.020,50.075.231,-33.304.513,-4.443.888,0,0,-4.141.136,-41.889.536,8.185.695,2.361.840,1.628.716,-3.246.402,-4.491.894,-995.624,5.717.578,6.033.312,-3.693.793,3.313.732,11.499.427,24.446,11.523.873,452.855,-5.066.434,6.910.293,0
BANCO PAN S.A.,59285411,BTG PACTUAL - PRUDENCIAL,49944,80336,b1,2,8,RIO DE JANEIRO,RJ,03/2024,2.865.058,0,216.490,204.014,214,78.995,3.364.771,-1.412.925,-19.633,0,0,-567.338,-1.999.897,1.364.874,135.274,195.571,-202.837,-875.434,-223.683,24.347,223.778,-416.535,-1.139.520,225.355,-697,224.658,-28.033,0,196.625,0
PICPAY BANK - BANCO MÚLTIPLO S.A,9516419,ORIGINAL - PRUDENCIAL,51781,80903,b1,2,8,SAO PAULO,SP,03/2024,318.004,0,262.801,0,0,0,580.806,-315.431,0,0,0,-53.280,-368.712,212.094,85.541,13.616,-42.265,2.681,-22.735,0,803,-84.073,-46.432,165.662,0,165.662,-58.586,-35.610,71.466,0
GOLDMAN SACHS DO BRASIL BANCO MULTIPLO S.A.,4332281,GOLDMAN SACHS - PRUDENCIAL,51585,80714,b2,3,8,SAO PAULO,SP,03/2024,10.503,0,457.756,70.397,0,0,538.656,-203.203,-129.818,0,-88.32,-1.660,-423.001,115.656,109.857,0,-110.289,-43.292,-10.788,9.623,24.226,-3.337,-24.001,91.655,787,92.441,-20.567,-4,71.870,0
REDECARD INSTITUIÇÃO DE PAGAMENTO S.A.,1425787,ITAU - PRUDENCIAL,,80099,n4,2,41,SAO PAULO,SP,03/2024,0,0,518.230,0,0,0,518.230,0,0,0,0,0,0,518.230,835.149,0,-114.827,-365.207,-115.586,14.716,782.401,-291.279,745.368,1.263.598,-1.706,1.261.892,-383.932,-105.273,772.687,0
SANTANDER LEASING S.A. ARRENDAMENTO MERCANTIL,47193149,SANTANDER - PRUDENCIAL,30379,80185,n1,3,19,SAO PAULO,SP,03/2024,537,419.772,73.696,0,0,0,494.006,-25.695,-672,-305.967,0,-1.180,-333.514,160.492,194,0,0,-1.819,-20.625,87.868,31.630,-15.387,81.862,242.353,2.257,244.611,-53.7,0,190.911,0
COOPERATIVA CENTRAL DE CRÉDITO DE MINAS GERAIS LTDA. - SICOOB CENTRAL CREDIMINAS,25683434,,,,b3C,2,9,BELO HORIZONTE,MG,03/2024,6.339,0,483.364,0,0,0,489.703,-472.776,-6.804,0,0,-32,-479.612,10.091,88,0,-8.289,-7.054,-4,26.329,14.022,-2.146,22.945,33.036,6,33.041,-18,-502,32.522,-19.094
COOPERATIVA CENTRAL DE CRÉDITO DE SANTA CATARINA E RIO GRANDE DO SUL - SICOOB CENTRAL SC/RS,80160260,,,,b3C,2,9,FLORIANOPOLIS,SC,03/2024,4.507,0,398.309,0,0,0,402.817,-399.154,-17,0,0,352,-398.818,3.998,16,0,-10.322,-9.241,-61,19.544,23.552,-563,22.925,26.923,20,26.943,-31,-1.216,25.696,0
HIPERCARD BANCO MÚLTIPLO S.A.,3012230,ITAU - PRUDENCIAL,10069,80099,b1,2,8,SAO PAULO,SP,03/2024,318.466,0,57.658,0,0,0,376.123,-36.933,-482,0,0,-298.776,-336.191,39.932,226.562,9.657,-3.886,-82.314,-31.358,2.586,11.122,-129.239,3.130,43.062,-51,43.011,-18.041,0,24.970,0
BANCO MUFG BRASIL S.A.,60498557,BCO MUFG BRASIL S.A. - PRUDENCIAL,,81713,b1,3,8,SAO PAULO,SP,03/2024,34.467,0,187.953,124.389,3.135,0,349.944,-88.512,-472.672,0,0,-113,-561.296,-211.352,4.868,96,-

24/07/17 19:00:47 WARN TaskSetManager: Stage 18 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


30


In [5]:
df_info_cadastral = G.table_to_df("finance_raw_data", "info_cadastral_entidades", spark)
df_info_cadastral.persist()

# dfSample_info_cadastral_entidades = df_info_cadastral.sample(False, 0.1).limit(30)
# table = tabulate(dfSample_info_cadastral_entidades.collect(), headers=dfSample_info_cadastral_entidades.columns, tablefmt='html')
# display(HTML(table))

# df_info_cadastral.count()
# dfSample_info_cadastral_entidades.printSchema()


DataFrame[data: string, codigo_cadastro_bacen: string, codigo_sisbacen: string, codigo_pais_sede: string, nome_pais_sede: string, nome_uf_sede: string, codigo_municipio_sede: string, nome_municipio_sede: string, nome_entidade: string, nome_entidade_nao_formatado: string, cnpj: string, cnpj_raiz: string, codigo_situacao: string, descricao_situacao: string, codigo_tipo_entidade_segmento: string, nome_tipo_entidade: string, codigo_natureza_juridica: string, descricao_natureza_juridica: string, codigo_esfera_publica: string, nome_reduzido: string, sigla_entidade: string, nome_fantasia: string, empresa_publica: string]

In [7]:
df_info_cadastral_select = df_info_cadastral.select("cnpj","codigo_sisbacen", "nome_tipo_entidade", "descricao_situacao", "nome_entidade", "nome_fantasia")

# a = df_info_cadastral_select.sample(False, 0.1).limit(50)
# table = tabulate(a.collect(), headers=a.columns, tablefmt='html')
# df_info_cadastral_select.filter(df_info_cadastral_select["cnpj"].isNull()).count()
# display(HTML(table))


In [8]:
df_join_ = df_info_cadastral_select.join(df_demonstracao_result, df_info_cadastral_select["nome_entidade"] == F.trim(df_demonstracao_result["instituicao"]), "full_outer")\
.select(df_demonstracao_result["instituicao"], df_demonstracao_result["tc"], df_demonstracao_result["conglomerado"], df_demonstracao_result["codigo"],\
        *df_info_cadastral_select.columns)

df_join = df_join_.dropDuplicates()

# Filtrar registros que estão apenas no DataFrame "demonstracao_result"
only_demonstracao_result = df_join.filter(
    F.col("nome_entidade").isNull() &
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "")
)

# Filtrar registros que estão apenas no DataFrame "info_cadastral"
only_info_cadastral = df_join.filter(
    F.col("instituicao").isNull() &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)

# Filtrar registros que estão presentes em ambos DataFrames
intersecao = df_join.filter(
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "") &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)
print(only_demonstracao_result.count())

print(only_info_cadastral.count())

print(intersecao.count())

print("total", df_join.count())


# testee = df_sem_acento.filter(F.col("nome_entidade") == "COOPERATIVA DE CRÉDITO RURAL COM INTERAÇÃO SOLIDÁRIA DE CASTRO - CRESOL CASTRO")

# table = tabulate(testee.collect(), headers=testee.columns, tablefmt='html')
# display(HTML(table))



24/07/17 19:01:09 WARN TaskSetManager: Stage 21 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:01:12 WARN TaskSetManager: Stage 22 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:01:13 WARN TaskSetManager: Stage 23 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


10


24/07/17 19:01:16 WARN TaskSetManager: Stage 36 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:01:17 WARN TaskSetManager: Stage 37 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


4032


24/07/17 19:01:18 WARN TaskSetManager: Stage 43 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:01:18 WARN TaskSetManager: Stage 44 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:01:19 WARN TaskSetManager: Stage 50 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


1561


24/07/17 19:01:19 WARN TaskSetManager: Stage 51 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


total 5603


In [9]:
sample_only_demonstracao_result = only_demonstracao_result.sample(False, 0.1).limit(30)

tb_only_demonstracao_result = tabulate(only_demonstracao_result.collect(), headers=only_demonstracao_result.columns, tablefmt='html')

sample_only_info_cadastral = only_info_cadastral.sample(False, 0.1).limit(30)

tb_only_info_cadastral = tabulate(sample_only_info_cadastral.collect(), headers=sample_only_info_cadastral.columns, tablefmt='html')

sample_intersecao =intersecao.sample(False, 0.1).limit(30)

tb_intersecao = tabulate(sample_intersecao.collect(), headers=sample_intersecao.columns, tablefmt='html')

display(HTML(tb_only_demonstracao_result))
display(HTML(tb_only_info_cadastral))
display(HTML(tb_intersecao))



24/07/17 19:02:41 WARN TaskSetManager: Stage 64 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:02:41 WARN TaskSetManager: Stage 65 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:02:42 WARN TaskSetManager: Stage 73 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:02:43 WARN TaskSetManager: Stage 74 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:02:44 WARN TaskSetManager: Stage 77 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 19:02:44 WARN TaskSetManager: Stage 78 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


instituicao,tc,conglomerado,codigo,cnpj,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
COOPERATIVA DE CREDITO E ECONOMIA COM INTERAÇÃO SOLIDARIA - CRESOL SÃO JOÃO DO SUL,2,,9488496,,,,,,
SUDACRED - SOCIEDADE DE CRÉDITO AO MICROEMPREENDEDOR E À EMPRESA DE PEQUENO PORTE LTDA.,2,SUDACRED SCD - PRUDENCIAL,20251847,,,,,,
MAGNETIS - DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA,2,BTG PACTUAL - PRUDENCIAL,87963450,,,,,,
CREFAZ SOCIEDADE DE CRÉDITO AO MICROEMPREENDEDOR E A EMPRESA DE PEQUENO PORTE LTDA.,2,CREFAZ SCMEPP LTDA - PRUDENCIAL,18188384,,,,,,
"BANIF - BANCO INTERNACIONAL DO FUNCHAL (BRASIL), S.A.",2,BANCO MASTER - PRUDENCIAL,33884941,,,,,,
FACILICRED SOCIEDADE DE CRÉDITO AO MICROEMPREENDEDOR LTDA,2,FACILICRED SCM LTDA - PRUDENCIAL,4849745,,,,,,
COOPERATIVA DE CRÉDITO E INVESTIMENTO COM INTERAÇÃO SOLIDÁRIA MEIO OESTE - CRESOL MEIO OESTE,2,,7946451,,,,,,
COOPERATIVA DE CRÉDITO SICOOB 3 COLINAS,2,,4013172,,,,,,
COOPERATIVA DE CRÉDITO RURAL DE PEQUENOS AGRICULTORES E DA REFORMA AGRÁRIA DO CENTRO OESTE DO PARANÁ - CREHNOR LARANJEIRAS,2,,1330387,,,,,,
"COOPERATIVA DE CREDITO, POUPANCA E INVESTIMENTO IGUACU - SICREDI IGUACU PR/SC/SP",2,,84974278,,,,,,


instituicao,tc,conglomerado,codigo,cnpj,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
,,,,62231287000118,,"Sociedade de Crédito, Financiamento e Investimento",Cancelada/Encerrada,"SAFIN S.A.CREDITO, FINANCIAMENTO E INVESTIMENTO",
,,,,08881641000104,,Cooperativa de Crédito,Cancelada/Encerrada,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS FUNCIONÁRIOS DA COMPANHIA DE CIMENTO PORTLAND POTY E EMPRESAS INTERLIGADAS LTDA,
,,,,03032915000177,54312,Cooperativa de Crédito,Cancelada/Encerrada,COOPERATIVA DE ECONOMIA E CREDITO MUTUO DOS SERVIDORES DO PODER EXECUTIVO DO ESTADO DE MATO GROSSO - COOPERFAZ,
,,,,62170824000167,,,Falta de integridade referencial na fonte,NOVACAO ADMINISTRACAO E PARTICIPACOES LTDA,NOVAÇÃO ASSET
,,,,76661842000156,,Sociedade Distribuidora de TVM,Cancelada/Encerrada,SENAP DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA,
,,,,92194794000190,,Administradora de Consórcio,Cancelada/Encerrada,COOPERATIVA DOS FUNCIONARIOS DO BANCO DO BRASIL LTDA,
,,,,36092302000184,,,Falta de integridade referencial na fonte,PRIME CCV EMPREENDIMENTOS E PARTICIPACOES S.A,
,,,,17015842000110,29971,Cooperativa de Crédito,Cancelada/Encerrada,COOPERATIVA DE CRÉDITO RURAL DE ECONOMIA SOLIDÁRIA DE CODAJÁS - CODCRED,
,,,,26027326000104,,Sociedade Corretora de TVM,Cancelada/Encerrada,"AGM S.A. CORRETORA DE CÂMBIO, TÍTULOS E VALORES MOBILIÁRIOS",
,,,,02412022000194,,Cooperativa de Crédito,Cancelada/Encerrada,"COOPERATIVA DE CRÉDITO DOS PROFISSIONAIS DA ÁREA DA SAÚDE DE FRANCISCO BELTRÃO-PR E REGIÃO, UNIPRIME FRANCISCO BELTRÃO",UNIPRIME FRANCISCO BELTRÃO


instituicao,tc,conglomerado,codigo,cnpj,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
AGENCIA DE FOMENTO DO AMAPA S.A.,1,AG. FOMENTO AMAPA S.A. - PRUDENCIAL,2929977,02929977000113,13157,Agência de Fomento,Autorizada em Atividade,AGENCIA DE FOMENTO DO AMAPA S.A.,AGÊNCIA DE FOMENTO POPULAR
"RPW S/A SOCIEDADE DE CRÉDITO, FINANCIAMENTO E INVESTIMENTO",2,RPW S/A SCFI - PRUDENCIAL,6249129,06249129000114,18592,"Sociedade de Crédito, Financiamento e Investimento",Autorizada em Atividade,"RPW S/A SOCIEDADE DE CRÉDITO, FINANCIAMENTO E INVESTIMENTO",EMPRESTA CAPITAL
U4C INSTITUIÇÃO DE PAGAMENTO S.A.,2,U4C IP - PRUDENCIAL,30980539,30980539000115,00029,Instituição de Pagamento,Autorizada em Atividade,U4C INSTITUIÇÃO DE PAGAMENTO S.A.,U4C INSTITUIÇÃO DE PAGAMENTO S.A.
FRIDAY INSTITUIÇÃO DE PAGAMENTOS LTDA.,2,FRIDAY IP - PRUDENCIAL,42926025,42926025000160,00010,Instituição de Pagamento,Autorizada em Atividade,FRIDAY INSTITUIÇÃO DE PAGAMENTOS LTDA.,FRIDAY IP
BANCO LOSANGO S.A. - BANCO MÚLTIPLO,2,BRADESCO - PRUDENCIAL,33254319,33254319008420,31168,Banco Múltiplo,Autorizada em Atividade,BANCO LOSANGO S.A. - BANCO MÚLTIPLO,BANCO LOSANGO S.A.
FINAXIS CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,2,FINAXIS - PRUDENCIAL,3317692,03317692000194,09083,Sociedade Corretora de TVM,Autorizada em Atividade,FINAXIS CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,FINAXIS CTVM
BRADESCO-KIRTON CORRETORA DE CÂMBIO S.A.,2,BRADESCO - PRUDENCIAL,58229246,58229246000110,57480,Sociedade Corretora de Câmbio,Autorizada em Atividade,BRADESCO-KIRTON CORRETORA DE CÂMBIO S.A.,BRADESCO-KIRTON CTVM
BANCO MIZUHO DO BRASIL S.A.,3,MIZUHO - PRUDENCIAL,61088183,61088183000133,06370,Banco Múltiplo,Autorizada em Atividade,BANCO MIZUHO DO BRASIL S.A.,BANCO MIZUHO
RENDIMENTOPAY INSTITUIÇÃO DE PAGAMENTO S.A.,2,RENDIMENTO - PRUDENCIAL,13776742,13776742000155,42886,Instituição de Pagamento,Autorizada em Atividade,RENDIMENTOPAY INSTITUIÇÃO DE PAGAMENTO S.A.,RENDIMENTOPAY
CORPORE SOCIEDADE DE CRÉDITO DIRETO S.A.,2,CORPORE SCD - PRUDENCIAL,7327045,07327045000114,19383,Sociedade de Crédito Direto,Autorizada em Atividade,CORPORE SOCIEDADE DE CRÉDITO DIRETO S.A.,CORPORE INVEST


In [19]:
df_group_situacao = only_info_cadastral.groupBy("nome_tipo_entidade").agg(F.count("*").alias("freq"))

# df_group_situacao.show()
# desc_sit = only_info_cadastral.filter(F.col("descricao_situacao") == "Autorizada em Atividade").limit(50)

table = tabulate(df_group_situacao.collect(), headers=df_group_situacao.columns, tablefmt='html')
display(HTML(table))


24/07/17 20:14:23 WARN TaskSetManager: Stage 113 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/17 20:14:23 WARN TaskSetManager: Stage 114 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


nome_tipo_entidade,freq
Banco de Desenvolvimento,8
Caixa Econômica Estadual,4
Associação de Poupança e Empréstimo,2
Sociedade de Crédito Imobiliário - Repassadora,10
,340
Sociedade de Crédito Imobiliário,78
Falta de integridade referencial na fonte,21
Administradora de Fundos não sujeita a autorização pelo BCB,10
Sociedade de Crédito Direto,22
"Sociedade de Crédito, Financiamento e Investimento",121


### Remover acentos

In [ ]:
def clean_accent_(texto):
    return unidecode(texto) if texto else None

clean_accent = F.udf(clean_accent_, F.StringType())

df_sem_acento = df_join.withColumn("nome_tipo_entidade", F.upper(clean_accent(F.col("nome_tipo_entidade"))))\
                                     .withColumn("descricao_situacao", F.upper(clean_accent(F.col("descricao_situacao"))))\
                                     .withColumn("nome_entidade", F.upper(F.trim(clean_accent(F.col("nome_entidade")))))\
                                    .withColumn("nome_fantasia", F.upper(clean_accent(F.col("nome_fantasia"))))


# table = tabulate(teste.collect(), headers=teste.columns, tablefmt='html')
# # display(HTML(table))
# table2 = tabulate(df_sem_acento.collect(), headers=df_sem_acento.columns, tablefmt='html')
# # display(HTML(table2))
